# TRABAJO FINAL - DATA VIZ

## PARTE 1 - EDA

* Escobar, Aldo
* Alvarez Muñoz, Carolina

El dataset a analizar consiste en 59946 perfiles de usuario de la aplicacion de citas online, OkCupid, El mismo contiene los siguientes campos:
* age
* body_type
* diet 
* drinks 
* drugs 
* education
* ethnicity 
* height 
* income 
* job 
* last_online 
* location 
* offspring 
* orientation
* essay0 
* essay1
* essay2
* essay3
* essay4
* essay5
* essay6
* essay7
* essay8 
* essay9
* location
* pets 
* religion 
* sex 
* sign 
* smokes 
* speaks 
* status

Para el trabajo se utilizaron la mayoria de estos campos (no todos). 

El trabajo se compone de 3 partes:
* Análisis de datos en general del data set
* Análisis de texto
* Análisis GIS

In [1]:
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import re
from statsmodels.graphics.mosaicplot import mosaic
import matplotlib.pyplot as plt
from plotly import tools
from sklearn import preprocessing

# Funciones para limpieza de texto

In [2]:
def normalize(string):
    return re.sub("[^0-9a-zA-Z]+", " ", string.lower())

In [3]:
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  cleantext = cleantext.replace('\n', ' ')
  return cleantext

# Lectura del data set y preprocesamiento de datos a utilizar

In [4]:
df=pd.read_csv("profiles.csv")
df=df.fillna(({'religion':'missing', 'drugs':'missing', 'drinks':'missing', 'ethnicity':'missing', 'job': 'missing', 
               'essay0':'','essay1':'','essay2':'','essay3':'','essay4':'','essay5':'',
               'essay6':'','essay7':'','essay8':'','essay9':'', 'smokes':'missing', 'body_type': 'missing'}))



for i in range(10):
    df["essay"+str(i)] = df["essay"+str(i)].apply(cleanhtml)
    df["essay"+str(i)] = df["essay"+str(i)].apply(normalize)
    df["essay"+str(i)] = df["essay"+str(i)].str.len()

df["essay_char_len"] = df.essay0 + df.essay1 + df.essay2 + df.essay3 + df.essay4 +  df.essay5 + df.essay6 + df.essay7 + df.essay8 + df.essay9    
for i in range(10):
    del df["essay"+str(i)]
    
    
    
lista_religion=[]
for r in list(df.religion):
    if "agnosticism" in str(r):
        lista_religion.append("agnosticim")
    if "atheism" in str(r):
        lista_religion.append("atheism")
    if "missing" in str(r):
        lista_religion.append("missing")
    if "christianity" in str(r):
        lista_religion.append("christianity")
    if "catholicism" in str(r):
        lista_religion.append("catholicism")
    if "buddhism" in str(r):
        lista_religion.append("buddhism")
    if "islam" in str(r):
        lista_religion.append("islam")
    if "other" in str(r):
        lista_religion.append("other")
    if "judaism" in str(r):
        lista_religion.append("judaism")
    if "hinduism" in str(r):
        lista_religion.append("hinduism")
        
df.religion = lista_religion
                             
df["height"] = df["height"].fillna(round(df["height"].mean(),0))
df["height"]=[round(df["height"].mean(),0) if x<50 else x for x in list(df["height"])]
df["height"]=df["height"]+100
df["age"]=[round(df["age"].mean(),0) if x>80 else x for x in list(df["age"])]

df.head(20)

,age,body_type,diet,drinks,drugs,education,ethnicity,height,income,job,...,offspring,orientation,pets,religion,sex,sign,smokes,speaks,status,essay_char_len
0,22.0,a little extra,strictly anything,socially,never,working on college/university,"asian, white",175.0,-1,transportation,...,"doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticim,m,gemini,sometimes,english,single,2386
1,35.0,average,mostly other,often,sometimes,working on space camp,white,170.0,80000,hospitality / travel,...,"doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticim,m,cancer,no,"english (fluently), spanish (poorly), french (...",single,1339
2,38.0,thin,anything,socially,missing,graduated from masters program,missing,168.0,-1,missing,...,NaN,straight,has cats,missing,m,pisces but it doesn&rsquo;t matter,no,"english, french, c++",available,5136
3,23.0,thin,vegetarian,socially,missing,working on college/university,white,171.0,20000,student,...,doesn&rsquo;t want kids,straight,likes cats,missing,m,pisces,no,"english, german (poorly)",single,415
4,29.0,athletic,NaN,socially,never,graduated from college/university,"asian, black, other",166.0,-1,artistic / musical / writer,...,NaN,straight,likes dogs and likes cats,missing,m,aquarius,no,english,single,622
5,29.0,average,mostly anything,socially,missing,graduated from college/university,white,167.0,-1,computer / hardware / software,...,"doesn&rsquo;t have kids, but might want them",straight,likes cats,atheism,m,taurus,no,"english (fluently), chinese (okay)",single,2244
6,32.0,fit,strictly anything,socially,never,graduated from college/university,"white, other",165.0,-1,missing,...,NaN,straight,likes dogs and likes cats,missing,f,virgo,missing,english,single,1770
7,31.0,average,mostly anything,socially,never,graduated from college/university,white,165.0,-1,artistic / musical / writer,...,"doesn&rsquo;t have kids, but wants them",straight,likes dogs and likes cats,christianity,f,sagittarius,no,"english, spanish (okay)",single,1154
8,24.0,missing,strictly anything,socially,missing,graduated from college/university,white,167.0,-1,missing,...,doesn&rsquo;t have kids,straight,likes dogs and likes cats,christianity,f,gemini but it doesn&rsquo;t matter,when drinking,english,single,2079
9,37.0,athletic,mostly anything,not at all,never,working on two-year college,white,165.0,-1,student,...,NaN,straight,likes dogs and likes cats,atheism,m,cancer but it doesn&rsquo;t matter,no,english (fluently),single,2097


# Division del data set y análisis de visualización por sexo

In [5]:
df_m = df[df.sex=="m"]
df_f = df[df.sex=="f"]

In [6]:
data = [go.Bar(
            x=['male'],
            y=[df.groupby("sex").get_group("m").count()[0]],
              marker=dict(color='cornflowerblue'),name="male"),
        go.Bar(
            x=['female'],
            y=[df.groupby("sex").get_group("f").count()[0]],
              marker=dict(color='darksalmon'),name="female")
             ]
layout = dict(title = 'Cantidad de hombres y mujeres',
              xaxis = dict(title = 'Sexo'),
              yaxis = dict(title = 'Cantidad')
              )
fig = dict(data=data, layout=layout)
iplot(fig)

In [7]:
# Create and style traces
trace0 = go.Scatter(
    x = df_m.groupby("age").count()["religion"].index,
    y = list(df_m.groupby("age").count()["religion"]),
    name = 'Male',
    line = dict(
        color = 'cornflowerblue',
        width = 4)
)
trace1 = go.Scatter(
    x = df_f.groupby("age").count()["religion"].index,
    y = list(df_f.groupby("age").count()["religion"]),
    name = 'Female',
    line = dict(
        color = "darksalmon",
        width = 4)
)
data = [trace0, trace1]

# Edit the layout
layout = dict(title = 'Cantidad de hombres/mujeres por edad',
              xaxis = dict(title = 'Edad'),
              yaxis = dict(title = 'Cantidad'),
              )

fig = dict(data=data, layout=layout)
iplot(fig)

In [8]:
trace1 = go.Bar(
    x=sorted(list(df_m.age.unique())),
    y=[len(df_m[df_m.age==x])/len(df.loc[df.age == x])  for x in sorted(list(df_m.age.unique()))],
    marker=dict(color="cornflowerblue",
            opacity=0.6),name='Male'
)
trace2 = go.Bar(
    x=sorted(list(df_m.age.unique())),
    y=[len(df_f[df_f.age==x])/len(df.loc[df.age == x])   for x in sorted(list(df_m.age.unique()))],
    marker=dict(color="darksalmon",
            opacity=0.6),
    name='Female'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='stack', margin=go.layout.Margin(
        l=100,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    title = 'Proporción (%) de Hombres y Mujeres por Edad',
     xaxis = dict(title = 'Edad'),
    yaxis = dict(title= "Porcentaje", hoverformat = '.2%', tickformat = '%')
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [9]:
# Create and style traces
trace0 = go.Scatter(
    x = df_m.groupby("height").count()["religion"].index,
    y = list(df_m.groupby("height").count()["religion"]),
    name = 'Male',
    line = dict(
        color = 'cornflowerblue',
        width = 4)
)
trace1 = go.Scatter(
    x = df_f.groupby("height").count()["religion"].index,
    y = list(df_f.groupby("height").count()["religion"]),
    name = 'Female',
    line = dict(
        color = "darksalmon",
        width = 4)
)
data = [trace0, trace1]

# Edit the layout
layout = dict(title = 'Cantidad de hombres/mujeres por Altura',
              xaxis = dict(title = 'Altura'),
              yaxis = dict(title = 'Cantidad'),
              )

fig = dict(data=data, layout=layout)
iplot(fig)

In [10]:
jobs_m = sorted(list(set(df_m.job.unique()) - set(["missing", "unemployed"])), reverse=True)
jobs_f = sorted(list(set(df_f.job.unique()) - set(["missing", "unemployed"])), reverse=True)
trace1 = {"x": [df_m[(df_m.job == x) & (df_m["income"]>0)]["income"].mean()  for x in jobs_m], 
          "y": jobs_m, 
          "marker": {"color": 'cornflowerblue', "size": 12}, 
          "mode": "markers", 
          "name": "Male", 
          "type": "scatter"
}

trace2 = {"x": [df_f[(df_f.job == x) & (df_f["income"]>0)]["income"].mean()  for x in jobs_f], 
          "y": jobs_f, 
          "marker": {"color": "darksalmon", "size": 12}, 
          "mode": "markers", 
          "name": "Female", 
          "type": "scatter", 
}

data = [trace1, trace2]
layout = dict(title= "Disparidad en ganancias según genero", 
          xaxis= {"title": "Ingresos", }, 
         margin=go.layout.Margin(
        l=220,
        r=50,
        b=100,
        t=50,
        pad=4))

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [11]:
body_type_list = sorted(list(set(df.body_type.unique()) - set(["missing"])), reverse=True)
trace1 = go.Bar(
    y=body_type_list,
    x=[len(df_m[df_m.body_type==x])  for x in body_type_list],
    marker=dict(color="cornflowerblue",
            opacity=0.6),name='Male',orientation='h'
)
trace2 = go.Bar(
    y=body_type_list,
    x=[len(df_f[df_f.body_type==x])  for x in body_type_list],
    marker=dict(color="darksalmon",
            opacity=0.6),
    name='Female',orientation='h'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='group', margin=go.layout.Margin(
        l=120,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    title = 'Cantidad de Hombres y Mujeres por Tipo de Cuerpo',
              xaxis = dict(title = 'Cantidad')
              
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

# Division del data set y visualización de datos segun tipo de carreras (STEM y no STEM)

In [12]:
df_stem = df[df.job.isin(["science / tech / engineering", "computer / hardware / software"])]
df_noStem= df[df.job.isin(['transportation', 'hospitality / travel', 'artistic / musical / writer',
               'banking / financial / real estate', 'entertainment / media',
               'sales / marketing / biz dev', 'medicine / health',  'executive / management',
               'education / academia', 'clerical / administrative',
               'construction / craftsmanship', 'political / government', 'law / legal services', 'military'])]

In [13]:
trace1 = go.Bar(
    y=jobs_m,
    x=[len(df_m[df_m.job==x])  for x in jobs_m],
    marker=dict(color="cornflowerblue",
            opacity=0.6),name='Male',orientation='h'
)
trace2 = go.Bar(
    y=jobs_f,
    x=[len(df_f[df_f.job==x])  for x in jobs_f],
    marker=dict(color="darksalmon",
            opacity=0.6),
    name='Female',orientation='h'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='stack', margin=go.layout.Margin(
        l=250,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    title = 'Cantidad de Hombres y Mujeres por Ocupación',
              xaxis = dict(title = 'Cantidad')
              
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [14]:
jobs_m = sorted(list(set(df_m.job.unique()) - set("missing")), reverse=True)
jobs_f = sorted(list(set(df_f.job.unique()) - set("missing")), reverse=True)

trace1 = go.Bar(
    y=jobs_m,
    x=[len(df_m[df_m.job==x])/len(df.loc[df.job == x])  for x in jobs_m],
    marker=dict(color="cornflowerblue",
            opacity=0.6),name='Male',orientation='h'
)
trace2 = go.Bar(
    y=jobs_f,
    x=[len(df_f[df_f.job==x])/len(df.loc[df.job == x])   for x in jobs_f],
    marker=dict(color="darksalmon",
            opacity=0.6),
    name='Female',orientation='h'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='stack', margin=go.layout.Margin(
        l=250,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    title = 'Proporción (%) de Hombres y Mujeres por Ocupación',
     xaxis = dict(title = 'Proporción (%)',
        hoverformat = '.2%', tickformat= '%'
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [15]:
data = [go.Bar(
            x=['STEM'],
            y=[df_stem.groupby("sex").get_group("f").count()[0]],
              marker=dict(color='darksalmon'),name="female stem"),
             go.Bar(
            
            x=['NO STEM'],
            y=[df_noStem.groupby("sex").get_group("f").count()[0]],
              marker=dict(color='rgb(222, 35, 91)'),name="female no stem", opacity=0.6),
       go.Bar(
           x=['STEM'],
            y=[df_stem.groupby("sex").get_group("m").count()[0]],
              marker=dict(color='cornflowerblue'),name="male stem"),
             go.Bar(
            x=['NO STEM'],
            y=[df_noStem.groupby("sex").get_group("m").count()[0]],
              marker=dict(color='rgb(35, 103, 222)'),name="male no stem", opacity=0.6)]

layout = go.Layout(title = "Cantidad de hombres y mujeres que vienen de carreras STEM y que no vienen de carreras STEM",
    barmode='stack', margin=go.layout.Margin(
        l=125,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    yaxis = dict(title="Cantidad"),
     xaxis = dict( title= "Tipo de carrera",
        hoverformat = '.2%'
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [16]:
data = [go.Bar(
            x=['Single STEM'],
            y=[df_stem[df_stem.status == "single"].groupby("sex").get_group("f").count()[0]],
              marker=dict(color='darksalmon'),name="female stem"),
             go.Bar(
            
            x=['Single NO STEM'],
            y=[df_noStem[df_noStem.status == "single"].groupby("sex").get_group("f").count()[0]],
              marker=dict(color='rgb(222, 35, 91)'),name="female no stem", opacity=0.6),
       go.Bar(
           x=['Single STEM'],
            y=[df_stem[df_stem.status == "single"].groupby("sex").get_group("m").count()[0]],
              marker=dict(color='cornflowerblue'),name="male stem"),
             go.Bar(
            x=['Single NO STEM'],
            y=[df_noStem[df_noStem.status == "single"].groupby("sex").get_group("m").count()[0]],
               marker=dict(color='rgb(35, 103, 222)'),name="male no stem", opacity=0.6)]

layout = go.Layout(title = "Cantidad de hombres y mujeres solteros/as que vienen de carreras STEM y que no vienen de carreras STEM",
    barmode='stack', margin=go.layout.Margin(
        l=125,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    yaxis = dict(title="Cantidad"),
     xaxis = dict( title= "Tipo de carrera",
        hoverformat = '.2%'
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [17]:
# Create and style traces
trace0 = go.Scatter(
    x = df_stem.groupby("age").sum().index,
    y = df_stem.groupby("age").sum()["essay_char_len"],
    name = 'STEM',
    line = dict(
        color = 'rgb(167, 103, 4)',
        width = 4)
)
trace1 = go.Scatter(
    x = df_noStem.groupby("age").sum().index,
    y = df_noStem.groupby("age").sum()["essay_char_len"],
    name = 'NO STEM',
    line = dict(
        color = "rgb(32, 205, 119)",
        width = 4)
)
data = [trace0, trace1]

# Edit the layout
layout = dict(title = 'Cantidad total de caracteres escritos por personas de carreras STEM y no STEM ordenado por edad',
              xaxis = dict(title = 'Edad'),
              yaxis = dict(title = 'Cantidad de caracteres'),
              )

fig = dict(data=data, layout=layout)
iplot(fig)

In [18]:
# Create and style traces
trace0 = go.Scatter(
    x = df_stem.groupby("age").mean().index,
    y = df_stem.groupby("age").mean()["essay_char_len"],
    name = 'STEM',
    line = dict(
        color = 'rgb(167, 103, 4)',
        width = 4)
)
trace1 = go.Scatter(
    x = df_noStem.groupby("age").mean().index,
    y = df_noStem.groupby("age").mean()["essay_char_len"],
    name = 'NO STEM',
    line = dict(
        color = "rgb(32, 205, 119)",
        width = 4)
)
data = [trace0, trace1]

# Edit the layout
layout = dict(title = 'Cantidad promedio de caracteres escritos por personas de carreras STEM y no STEM ordenado por edad',
              xaxis = dict(title = 'Edad'),
              yaxis = dict(title = 'Cantidad de caracteres'),
              )

fig = dict(data=data, layout=layout)
iplot(fig)

In [19]:
jobs_stem = sorted(list(set(df_stem.job.unique()) - set("missing")), reverse=True)
jobs_noStem = sorted(list(set(df_noStem.job.unique()) - set("missing")), reverse=True)

trace1 = {"x": [df_stem[(df_stem.job == x) & (df_stem["sex"]=="m")]["essay_char_len"].sum()  for x in jobs_stem], 
          "y": jobs_stem, 'opacity' : 0.5,
          "marker": {"color": 'rgb(167, 103, 4)', "size": 12,
                    "line":{'color': 'blue', 'width': 3}}, 
          "mode": "markers", 
          "name": "Male STEM", 
          "type": "scatter"
}

trace2 = {"x": [df_stem[(df_stem.job == x) & (df_stem["sex"]=="f")]["essay_char_len"].sum()  for x in jobs_stem], 
          "y": jobs_stem, 'opacity' : 0.5,
          "marker": {"color": 'rgb(167, 103, 4)', "size": 12,
                     "line":{'color': 'red', 'width': 3}}, 
          "mode": "markers", 
          "name": "Female STEM", 
          "type": "scatter"
}

trace3 = {"x": [df_noStem[(df_noStem.job == x) & (df_noStem["sex"]=="m")]["essay_char_len"].sum()  for x in jobs_noStem], 
          "y": jobs_noStem, 'opacity' : 0.5,
          "marker": {"color": "rgb(32, 205, 119)", "size": 12,
                    "line":{'color': 'blue', 'width': 3}}, 
          "mode": "markers", 
          "name": "Male NO STEM", 
          "type": "scatter", 
}


trace4 = {"x": [df_noStem[(df_noStem.job == x) & (df_noStem["sex"]=="f")]["essay_char_len"].sum()  for x in jobs_noStem], 
          "y": jobs_noStem, 'opacity' : 0.5,
          "marker": {"color": 'rgb(32, 205, 119)', "size": 12,
                     "line":{'color': 'red', 'width': 3}},
          "mode": "markers", 
          "name": "Female NO STEM", 
          "type": "scatter"
}


data = [trace1, trace2,trace3,trace4]
layout = dict(title= "Disparidad en cantidad total de caracteres escritos en ensayos según tipo de carrera y sexo", 
          xaxis= {"title": "Cantidad total de caracteres escritos"}, 
         margin=go.layout.Margin(
        l=220,
        r=50,
        b=100,
        t=50,
        pad=4))

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [20]:
jobs_stem = sorted(list(set(df_stem.job.unique()) - set("missing")), reverse=True)
jobs_noStem = sorted(list(set(df_noStem.job.unique()) - set("missing")), reverse=True)

trace1 = {"x": [df_stem[(df_stem.job == x) & (df_stem["sex"]=="m")]["essay_char_len"].mean()  for x in jobs_stem], 
          "y": jobs_stem, 'opacity' : 0.5,
          "marker": {"color": 'rgb(167, 103, 4)', "size": 12,
                    "line":{'color': 'blue', 'width': 3}}, 
          "mode": "markers", 
          "name": "Male STEM", 
          "type": "scatter"
}

trace2 = {"x": [df_stem[(df_stem.job == x) & (df_stem["sex"]=="f")]["essay_char_len"].mean()  for x in jobs_stem], 
          "y": jobs_stem, 'opacity' : 0.5,
          "marker": {"color": 'rgb(167, 103, 4)', "size": 12,
                     "line":{'color': 'red', 'width': 3}}, 
          "mode": "markers", 
          "name": "Female STEM", 
          "type": "scatter"
}

trace3 = {"x": [df_noStem[(df_noStem.job == x) & (df_noStem["sex"]=="m")]["essay_char_len"].mean()  for x in jobs_noStem], 
          "y": jobs_noStem, 'opacity' : 0.5,
          "marker": {"color": "rgb(32, 205, 119)", "size": 12,
                    "line":{'color': 'blue', 'width': 3}}, 
          "mode": "markers", 
          "name": "Male NO STEM", 
          "type": "scatter", 
}


trace4 = {"x": [df_noStem[(df_noStem.job == x) & (df_noStem["sex"]=="f")]["essay_char_len"].mean()  for x in jobs_noStem], 
          "y": jobs_noStem, 'opacity' : 0.5,
          "marker": {"color": 'rgb(32, 205, 119)', "size": 12,
                     "line":{'color': 'red', 'width': 3}},
          "mode": "markers", 
          "name": "Female NO STEM", 
          "type": "scatter"
}


data = [trace1, trace2,trace3,trace4]
layout = dict(title= "Disparidad en cantidad promedio de caracteres escritos en ensayos según tipo de carrera y sexo", 
          xaxis= {"title": "Cantidad promedio de caracteres escritos"}, 
         margin=go.layout.Margin(
        l=220,
        r=50,
        b=100,
        t=50,
        pad=4))

fig = go.Figure(data=data, layout=layout)
iplot(fig)

# Boxplots de religiones

In [21]:
data = [go.Box(
    y=df[(df.income>0) & (df.religion==x)]['income'], name = x)  for x in list(set(df.religion.unique()) -set(["missing", "other"]))]


layout = dict(title= "Boxplot de Ingresos para cada religión", 
          xaxis= {"title": "Religiones"}, 
        yaxis= {"title": "Ingresos"})


fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [22]:
data = [go.Box(
    y=df[(df.religion==x)]['essay_char_len'], name = x)  for x in list(set(df.religion.unique()) -set(["missing", "other"]))]


layout = dict(title= "Boxplot de Caracteres escritos para cada religión", 
          xaxis= {"title": "Religiones"}, 
        yaxis= {"title": "Caracteres escritos"})


fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [23]:
data = [go.Box(
    y=df[(df.religion==x)]['age'], name = x)  for x in list(set(df.religion.unique()) -set(["missing", "other"]))]


layout = dict(title= "Boxplot de edad para cada religión", 
          xaxis= {"title": "Religiones"}, 
        yaxis= {"title": "Edad"})


fig = go.Figure(data=data, layout=layout)
iplot(fig)

# Analisis de habito de consumo de alcohol de ambos sexos según edad, religion y situación de fumador

In [24]:
i=0 
data=[]
colors = ["green", "grey", "blue", "yellow", "pink", "red"]
for d in ['not at all', 'rarely', 'socially', 'often', 'very often', 'desperately']:
    data.append(go.Bar(
    y=sorted(list(df_m[df_m["drinks"] == d ].age.unique())),
    x=[len(df_m[df_m["drinks"] == d ][df_m[df_m["drinks"] == d ].age==x])/len(df_m.loc[(df_m.age == x) & (df_m.drinks != "missing")])   for x in sorted(list(df_m[df_m["drinks"] == d ].age.unique()))],
    marker=dict(color=colors[i],
            opacity=0.6),
    name='Male drinks - '+d,orientation='h'))
    i+=1
    
layout = go.Layout(
    barmode='stack', margin=go.layout.Margin(
        l=100,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    title = 'Cambio de la proporción del habito de consumo de alcohol de hombres según la edad ',
     xaxis = dict(title = 'Proporción (%) de cada situación de alcohol',
        hoverformat = '.2%', tickformat="%"
    ),
    yaxis = dict(title="Edad")
)
fig = go.Figure(data=data,layout=layout)
iplot(fig)

In [25]:
i=0 
data=[]
for d in ['not at all', 'rarely', 'socially', 'often', 'very often', 'desperately']:
    data.append(go.Bar(
    y=sorted(list(df_f[df_f["drinks"] == d ].age.unique())),
    x=[len(df_f[df_f["drinks"] == d ][df_f[df_f["drinks"] == d ].age==x])/len(df_f.loc[(df_f.age == x) & (df_f.drinks != "missing")])   for x in sorted(list(df_f[df_f["drinks"] == d ].age.unique()))],
    marker=dict(color=colors[i],
            opacity=0.6),
    name='Female drinks - '+d,orientation='h'))
    i+=1
    
layout = go.Layout(
    barmode='stack', margin=go.layout.Margin(
        l=100,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    title = 'Cambio de la proporción del habito de consumo de alcohol de mujeres según la edad ',
     xaxis = dict(title = 'Proporción (%) de cada situación de alcohol',
        hoverformat = '.2%', tickformat="%"
    ),
    yaxis = dict(title="Edad")
)
fig = go.Figure(data=data,layout=layout)
iplot(fig)

In [26]:
i=0 
data=[]
colors = ["green", "grey", "blue", "yellow", "pink", "red"]
for d in ['not at all', 'rarely', 'socially', 'often', 'very often', 'desperately']:
    data.append(go.Bar(
    y=sorted(list(df_m[df_m["drinks"] == d ].religion.unique())),
    x=[len(df_m[df_m["drinks"] == d ][df_m[df_m["drinks"] == d ].religion==x])/len(df_m.loc[(df_m.religion == x) & (df_m.drinks != "missing")])   for x in sorted(list(df_m[df_m["drinks"] == d ].religion.unique()))],
    marker=dict(color=colors[i],
            opacity=0.6),
    name='Male drinks - '+d,orientation='h'))
    i+=1
    
layout = go.Layout(
    barmode='stack', margin=go.layout.Margin(
        l=100,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    title = 'Cambio de la proporción del habito de consumo de alcohol de hombres según religión',
     xaxis = dict(title = 'Proporción (%) de cada situación de alcohol',
        hoverformat = '.2%', tickformat="%"
    ),
    yaxis = dict(title="Religion")
)
fig = go.Figure(data=data,layout=layout)
iplot(fig)

In [27]:
i=0 
data=[]
colors = ["green", "grey", "blue", "yellow", "pink", "red"]
for d in ['not at all', 'rarely', 'socially', 'often', 'very often', 'desperately']:
    data.append(go.Bar(
    y=sorted(list(df_f[df_f["drinks"] == d ].religion.unique())),
    x=[len(df_f[df_f["drinks"] == d ][df_f[df_f["drinks"] == d ].religion==x])/len(df_f.loc[(df_f.religion == x) & (df_f.drinks != "missing")])   for x in sorted(list(df_f[df_f["drinks"] == d ].religion.unique()))],
    marker=dict(color=colors[i],
            opacity=0.6),
    name='Female drinks - '+d,orientation='h'))
    i+=1
    
layout = go.Layout(
    barmode='stack', margin=go.layout.Margin(
        l=100,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    title = 'Cambio de la proporción del habito de consumo de alcohol de mujeres según religión',
     xaxis = dict(title = 'Proporción (%) de cada situación de alcohol',
        hoverformat = '.2%', tickformat="%"
    ),
    yaxis = dict(title="Religion")
)
fig = go.Figure(data=data,layout=layout)
iplot(fig)

In [28]:
i=0 
data=[]
colors = ["green", "grey", "blue", "yellow", "pink", "red"]
for d in ['not at all', 'rarely', 'socially', 'often', 'very often', 'desperately']:
    data.append(go.Bar(
    y=sorted(list(df_m[df_m["drinks"] == d ].smokes.unique())),
    x=[len(df_m[df_m["drinks"] == d ][df_m[df_m["drinks"] == d ].smokes==x])/len(df_m.loc[(df_m.smokes == x) & (df_m.drinks != "missing")])   for x in sorted(list(df_m[df_m["drinks"] == d ].smokes.unique()))],
    marker=dict(color=colors[i],
            opacity=0.6),
    name='Male drinks - '+d,orientation='h'))
    i+=1
    
layout = go.Layout(
    barmode='stack', margin=go.layout.Margin(
        l=200,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    title = 'Cambio de la proporción del habito de consumo de alcohol de hombres según situacion de fumador',
     xaxis = dict(title = 'Proporción (%) de cada situación de alcohol',
        hoverformat = '.2%', tickformat="%"
    ),
    yaxis = dict(title = 'Situacion de Fumador')
)
fig = go.Figure(data=data,layout=layout)
iplot(fig)

In [29]:
i=0 
data=[]
colors = ["green", "grey", "blue", "yellow", "pink", "red"]
for d in ['not at all', 'rarely', 'socially', 'often', 'very often', 'desperately']:
    data.append(go.Bar(
    y=sorted(list(df_f[df_f["drinks"] == d ].smokes.unique())),
    x=[len(df_f[df_f["drinks"] == d ][df_f[df_f["drinks"] == d ].smokes==x])/len(df_f.loc[(df_f.smokes == x) & (df_f.drinks != "missing")])   for x in sorted(list(df_f[df_f["drinks"] == d ].smokes.unique()))],
    marker=dict(color=colors[i],
            opacity=0.6),
    name='Female drinks - '+d,orientation='h'))
    i+=1
    
layout = go.Layout(
    barmode='stack', margin=go.layout.Margin(
        l=200,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    title = 'Cambio de la proporción del habito de consumo de alcohol de mujeres según situacion de fumador',
     xaxis = dict(title = 'Proporción (%) de cada situación de alcohol',
        hoverformat = '.2%', tickformat="%"
    ),
    yaxis = dict(title = 'Situacion de Fumador')
)
fig = go.Figure(data=data,layout=layout)
iplot(fig)

# Normalización de datos numericos y visualización de los mismos en un Scatter Polar

In [30]:
x_m = df_m[['age','height','income', 'essay_char_len']][df_m.income >0].values #returns a numpy array
x_f = df_f[['age','height','income', 'essay_char_len']][df_f.income >0].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_m_scaled = min_max_scaler.fit_transform(x_m)
x_f_scaled = min_max_scaler.fit_transform(x_f)
df_m_z = pd.DataFrame(x_m_scaled, columns = ['age','height','income', 'essay_char_len'])
df_f_z = pd.DataFrame(x_f_scaled, columns = ['age','height','income', 'essay_char_len'])


In [31]:
data = [
    go.Scatterpolar(
      r = [df_m_z[col].mean() for col in ['age','height','income', 'essay_char_len', 'age']],
      theta = ['Edad','Altura','Ingresos', 'Cantidad de caracteres escritos','Edad'],
      fill = 'toself',
      name = 'Male',
      marker = {"color": 'cornflowerblue'}
    ),
    go.Scatterpolar(
      r = [df_f_z[col].mean() for col in ['age','height','income', 'essay_char_len', 'age']],
      theta = ['Edad','Altura','Ingresos', 'Cantidad de caracteres escritos','Edad'],
      fill = 'toself',
      name = 'Female',
    marker = {"color": 'darksalmon'}
    )
]

layout = go.Layout( title= "Radar graph de Edad, Altura, Ingresos y Cantidad de caracteres escritos según sexo",
  polar = dict(
    radialaxis = dict(
      visible = True,
      range = [0, 0.5]
    )
  ),
  showlegend = True
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [32]:
x_stem = df_stem[['age','height','income', 'essay_char_len']][df_stem.income >0].values #returns a numpy array
x_noStem = df_noStem[['age','height','income', 'essay_char_len']][df_noStem.income >0].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_stem_scaled = min_max_scaler.fit_transform(x_stem)
x_noStem_scaled = min_max_scaler.fit_transform(x_noStem)
df_stem_z = pd.DataFrame(x_stem_scaled, columns = ['age','height','income', 'essay_char_len'])
df_noStem_z = pd.DataFrame(x_noStem_scaled, columns = ['age','height','income', 'essay_char_len'])

In [33]:
data = [
    go.Scatterpolar(
      r = [df_stem_z[col].mean() for col in ['age','height','income', 'essay_char_len', 'age']],
      theta = ['Edad','Altura','Ingresos', 'Cantidad de caracteres escritos','Edad'],
      fill = 'toself',
      name = 'STEM',
      marker = {"color": 'rgb(167, 103, 4)'}
    ),
    go.Scatterpolar(
      r = [df_noStem_z[col].mean() for col in ['age','height','income', 'essay_char_len', 'age']],
      theta = ['Edad','Altura','Ingresos', 'Cantidad de caracteres escritos','Edad'],
      fill = 'toself',
      name = 'NO STEM',
    marker = {"color": "rgb(32, 205, 119)"}
    )
]

layout = go.Layout( title= "Radar graph de Edad, Altura, Ingresos y Cantidad de caracteres escritos según tipo de carrera",
  polar = dict(
    radialaxis = dict(
      visible = True,
      range = [0, 0.5]
    )
  ),
  showlegend = True
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)